In [1]:
import os
import time
import yaml
import math
import pandas as pd
import numpy as np

In [2]:
from data.excel2pd import data_load
from data.preprocessing import data_preprocessing
from data.train_test_split import data2Window, X_Y2train_test, X_Y_from_data
from data.CustomDataset import CustomDataset

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

import sys
sys.setrecursionlimit(4000)

In [4]:
# torch version, gpu 사용 여부, 이름 체크
print("torch version>>", torch.__version__)
print("GPU 사용 유무>>", torch.cuda.is_available())
if torch.cuda.is_available() :
    print("GPU 이름>> ", torch.cuda.get_device_name(0))
    device = "cuda:0" 
else :
    device = "cpu"
print(device)

torch version>> 1.13.1
GPU 사용 유무>> True
GPU 이름>>  NVIDIA GeForce RTX 2080
cuda:0


In [5]:
f = open("config.yaml", 'r')
config = yaml.load(f, Loader=yaml.FullLoader)

data = data_load(11)
data, scaler = data_preprocessing(data, config['data_options'])

X, Y = X_Y_from_data(data)

X_train, Y_train, X_test, Y_test = X_Y2train_test(X=X, Y=Y,
                                                  input_window=config['input_window'],
                                                  output_window=config['output_window'],
                                                  feature=config['data_options']['feature'],
                                                  stride=config['stride'])

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(45866, 36, 1) (45866, 6)
(279, 36, 1) (279, 6)


In [6]:
dataset = CustomDataset(X_train, Y_train)
dataloader = DataLoader(dataset, batch_size = config['batch'], shuffle=False)

In [7]:
from model.model import univariate_TSTransformer
from model.positional_encoding import FixedPositionalEncoding
from model.train import train

In [8]:
model = univariate_TSTransformer(d_model = 512, 
                                 input_window = config['input_window'] + config['output_window'], 
                                 output_window = config['output_window'], 
                                 nhead = 8, 
                                 num_layers = 6, 
                                 dropout = 0.1).to(device)

In [9]:
model, optimizer = train(device, model, dataloader, config["learning_rate"], config['epoch'], config['output_window'])

  0%|          | 0/100 [00:00<?, ?it/s]

model output shape :  torch.Size([128, 6])
label shape :  torch.Size([128, 6])
model output shape :  torch.Size([128, 6])
label shape :  torch.Size([128, 6])
model output shape :  torch.Size([128, 6])
label shape :  torch.Size([128, 6])


  0%|          | 0/100 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
result = model.predict(torch.Tensor(X_test).to(device)).to(device)
result = result.cpu().detach().numpy()

In [ ]:
#model save
import os
from datetime import datetime
#from model.train import save_model

now = datetime.now()
save_path = "./result/"+now.strftime('%Y_%m_%d_%H_%M')
try:
    if not os.path.exists(save_path) : 
        os.mkdir(save_path)
except:
    print("Error : Failed to create the directory.")

In [ ]:
from model.result import result_jpg, save_yaml_and_error, save_numeric_result

result_jpg(result, Y_test, save_path)
save_yaml_and_error("./", save_path, now)
save_numeric_result(result, Y_test, now)